### Create the jsonl for the gork ag reasoning 300

In [ ]:
import os
import json

def load_processed_questions(file_path):
    """
    Reads the annotation JSON file and returns a list of all 'processed_question' strings.
    """
    with open(file_path, 'r') as f:
        data = json.load(f)
    return [entry['processed_question'] for entry in data]

if __name__ == "__main__":
    # === CONFIGURABLE PARAMETERS ===
    model_name = "grok-3-beta"  # ← change this to target any other model
    annotation_file = (
        "/work/mech-ai-scratch/zare/Prompt_creation/answer-creation-grok/"
        "data/final_annotation_qa.json"
    )
    output_base = "/work/mech-ai-scratch/zare/Prompt_creation/answer-creation-grok/data"
    # ===============================

    # Load questions
    questions = load_processed_questions(annotation_file)

    # Build tasks list
    tasks = []
    for idx, q in enumerate(questions, start=1):
        tasks.append({
            "model": model_name,
            "metadata": {"question_index": idx},
            "messages": [
                {
                    "role": "system",
                    "content": (
                        "You are an expert agronomy assistant. Provide clear, complete, "
                        "and actionable advice on cover crop and abiotic stress recommendations."
                    )
                },
                {
                    "role": "user",
                    "content": f"Please answer the following question:\n\n{q}"
                }
            ]
        })

    # Prepare output directory and file
    output_dir = os.path.join(output_base, model_name)
    os.makedirs(output_dir, exist_ok=True)
    output_file = os.path.join(output_dir, f"{model_name}_processed_questions.jsonl")

    # Write all tasks to a single JSONL
    with open(output_file, 'w') as out_f:
        for task in tasks:
            out_f.write(json.dumps(task) + "\n")

    print(f"Created {len(tasks)} tasks in:\n  {output_file}")


Created 297 tasks in:
  /work/mech-ai-scratch/zare/Prompt_creation/answer-creation-grok/data/grok-3-beta/grok-3-beta_processed_questions.jsonl


### Cleaning results

In [39]:
import json
model_name = "grok-3-beta"
def clean_results(jsonl_path, cleaned_path):
    cleaned = []
    with open(jsonl_path, 'r') as infile:
        for line in infile:
            # parse the three‐element array
            request_json, response_json, metadata = json.loads(line)

            # find the user message
            user_msg = next(m for m in request_json["messages"] if m["role"] == "user")
            raw = user_msg["content"]

            # extract the question text
            if isinstance(raw, str):
                question = raw
            else:
                question = " ".join(
                    item.get("text", "")
                    for item in raw
                    if isinstance(item, dict) and item.get("type") == "text"
                )

            # remove the unwanted prefix if present
            prefix = "Please answer the following question:\n\n"
            if question.startswith(prefix):
                question = question[len(prefix):]

            # extract the assistant answer
            answer = response_json["choices"][0]["message"]["content"]

            # response ID and any metadata
            resp_id = response_json.get("id")
            q_idx   = metadata.get("question_index")

            cleaned.append({
                "question_index": q_idx,
                "response_id":   resp_id,
                "question":      question,
                "answer":        answer
            })

    # write out the cleaned JSON
    with open(cleaned_path, 'w', encoding='utf-8') as out:
        json.dump(cleaned, out, indent=2, ensure_ascii=False)

if __name__ == "__main__":
    clean_results(
        jsonl_path=f"/work/mech-ai-scratch/zare/Prompt_creation/answer-creation-grok/data/{model_name}/{model_name}_processed_results.jsonl",
        cleaned_path=f"/work/mech-ai-scratch/zare/Prompt_creation/answer-creation-grok/data/{model_name}/{model_name}_cleaned_responses.json"
    )
    print("Done!")


Done!


### Getting the respons of the ground truth and the respons of one llm

In [5]:
import json
import pandas as pd

# Load the annotation file
with open('/work/mech-ai-scratch/zare/Prompt_creation/answer-creation-grok/final_annotation_qa.json', 'r') as f:
    annotations = json.load(f)

# Load the cleaned responses file
with open('/work/mech-ai-scratch/zare/Prompt_creation/answer-creation-grok/cleaned_responses.json', 'r') as f:
    cleaned = json.load(f)

# Map questions to cleaned answers
cleaned_map = {entry['question']: entry['answer'] for entry in cleaned}

# Find matching entries
matches = []
for ann in annotations:
    pq = ann.get('processed_question')
    ann_answer = ann.get('answer')
    if pq in cleaned_map:
        matches.append({
            'Question': pq,
            'DeepseekR1 Answer': ann_answer,
            'Grok 2 Answer': cleaned_map[pq]
        })

# Create DataFrame and print it
df = pd.DataFrame(matches)


In [8]:
df.to_csv('/work/mech-ai-scratch/zare/Prompt_creation/answer-creation-grok/matched_answers.csv', index=False, header=['Question', 'DeepseekR1 Answer', 'Grok 2 Answer']
)